In [ ]:
import sys
sys.path.append('/Users/vdk/GeneveWork/Code/calibpipe-main-calibpipe/')
from astropy.io import ascii
import numpy as np
import pandas as pd
import matplotlib
from seaborn import histplot
import csv
import scipy
import matplotlib.pyplot as plt
import random
import copy
from os import path, makedirs
from astropy.coordinates import SkyCoord
from astropy.table import Table
import astropy.units as u
from ctapipe.containers import MuonEfficiencyContainer
from ctapipe.coordinates import CameraFrame, TelescopeFrame
from ctapipe.io import EventSource, EventSeeker
from ctapipe.core import TelescopeComponent
from ctapipe.core.traits import (
    List,
    Int,
    FloatTelescopeParameter,
    TelescopeParameter,
    Unicode,
)
from ctapipe.image.cleaning import tailcuts_clean
from ctapipe.image.muon import MuonRingFitter, MuonIntensityFitter
from traitlets.config import Config
from ctapipe.image import ImageProcessor
from ctapipe.image.muon import MuonProcessor
from ctapipe.visualization import CameraDisplay
from ctapipe.instrument import CameraGeometry
from ctapipe.calib import CameraCalibrator
from traitlets.config.loader import Config, FileConfigLoader, JSONFileConfigLoader
import pathlib
from calibpipe.throughput import MuonCalibrator
from traitlets.config import Config
from astropy.time import Time
from astropy.coordinates import EarthLocation,SkyCoord, AltAz 
from traitlets.config import Config






In [ ]:
k = 6
event_number = 466
teln = 1
filename = f'/Users/vdk/Software/sim_run_folder/output_prod_v12/corsika_theta_28.4_az_182.75_run{k}.simtel.gz'
source = EventSource(filename, max_events=9000)
event_iterator = iter(source)

for i,j in enumerate(event_iterator):
    if i == event_number:
        image_processor = ImageProcessor(source.subarray)
        muon_processor = MuonProcessor(source.subarray)
        calib = CameraCalibrator(image_extractor_type="GlobalPeakWindowSum",subarray = source.subarray)
        calib(j)
        image_processor(j)
        muon_processor(j)
        event = j
        fig, axes = plt.subplots(2, 1, sharex=False, gridspec_kw={'height_ratios': [1, 1]}, figsize=(9,12))
        fig.subplots_adjust(hspace=0.2)
        ax = axes[0]
        camgeom = source.subarray.tel[1].camera.geometry
        title=f"event{event_number}_fit{round(j.muon.tel[teln].efficiency.optical_efficiency,3)}_width{round(j.muon.tel[teln].efficiency.width.to_value(),3)}"
        disp = CameraDisplay(camgeom,title=title, ax = ax)
        #disp.image = event.r1.tel[teln].waveform[:,11]
        #disp.image = event.dl0.tel[teln].waveform[:,11]
        disp.image = j.simulation.tel[teln].true_image #.sum(axis=1)
        disp.cmap = plt.cm.RdBu_r
        disp.add_colorbar()
        disp.set_limits_percent(95)
        bx = axes[1]
        for pix_id in range(0,1855):
            #bx.plot(j.r0.tel[teln].waveform[0][pix_id], label="pix {}".format(pix_id), drawstyle='steps')
            bx.plot(j.r1.tel[teln].waveform[pix_id], label="pix {}".format(pix_id), drawstyle='steps')
        #bx.set_ylabel("ADC counts")
        bx.set_xlabel("Samples")
        bx.set_title("Waveforms")
        plt.show()
        break




In [ ]:
k = 6
event_number = 466
teln = 1
filename = f'/Users/vdk/Software/sim_run_folder/output_prod_v12/corsika_theta_28.4_az_182.75_run{k}.simtel.gz'
source = EventSource(filename, max_events=9000)
event_iterator = iter(source)

for i,j in enumerate(event_iterator):
    if i == event_number:
        image_processor = ImageProcessor(source.subarray)
        muon_processor = MuonProcessor(source.subarray)
        calib = CameraCalibrator(image_extractor_type="GlobalPeakWindowSum",subarray = source.subarray)
        calib(j)
        image_processor(j)
        muon_processor(j)
        event = j
        fig, axes = plt.subplots(2, 1, sharex=False, gridspec_kw={'height_ratios': [1, 1]}, figsize=(9,12))
        fig.subplots_adjust(hspace=0.2)
        ax = axes[0]
        camgeom = source.subarray.tel[1].camera.geometry
        title=f"event{event_number}_fit{round(j.muon.tel[teln].efficiency.optical_efficiency,3)}_width{round(j.muon.tel[teln].efficiency.width.to_value(),3)}"
        disp = CameraDisplay(camgeom,title=title, ax = ax)
        #disp.image = event.r1.tel[teln].waveform[:,11]
        #disp.image = event.dl0.tel[teln].waveform[:,11]
        disp.image = j.simulation.tel[teln].true_image #.sum(axis=1)
        disp.cmap = plt.cm.RdBu_r
        disp.add_colorbar()
        disp.set_limits_percent(95)
        bx = axes[1]
        for pix_id in range(0,1855):
            #bx.plot(j.r0.tel[teln].waveform[0][pix_id], label="pix {}".format(pix_id), drawstyle='steps')
            bx.plot(j.r0.tel[teln].waveform[1][pix_id], label="pix {}".format(pix_id), drawstyle='steps')
        #bx.set_ylabel("ADC counts")
        bx.set_xlabel("Samples")
        bx.set_title("Waveforms")
        plt.show()
        break




In [ ]:
event.mon.tel[1].calibration.unusable_pixels

In [ ]:
n_pixels = [i for i in range(event.dl0.tel[teln].waveform.shape[-2])]
n_timesamples = [i for i in range(event.dl0.tel[teln].waveform.shape[-1])]
plt.scatter(event.dl1.tel[teln].peak_time, n_pixels, s = 10)
plt.title("Peak time for every pixel in dl1 data")
plt.xlabel("Pixel number")
plt.ylabel("Time sample of the peak")
#plt.savefig(f"/Users/vdk/PeakTime_DL1_event{str(k)+str(0)+str(event_number)}")

In [ ]:
n_pixels = [i for i in range(event.dl0.tel[teln].waveform.shape[-2])]
n_timesamples = [i for i in range(event.dl0.tel[teln].waveform.shape[-1])]
plt.scatter(n_pixels,event.dl1.tel[teln].peak_time, s = 10)
plt.title("Peak time for every pixel in dl1 data")
plt.xlabel("Pixel number")
plt.ylabel("Time sample of the peak")
plt.savefig(f"/Users/vdk/PeakTime_DL1_event{str(k)+str(0)+str(event_number)}")

In [ ]:
histplot(event.dl1.tel[teln].peak_time)
plt.xlabel("time sample")
plt.title("Hist of peak times for all pixels in dl1")

In [ ]:
peak_arr = np.empty(1855, dtype = int)
np.argmax(event.dl0.tel[teln].waveform, axis = 1, out = peak_arr)

In [ ]:
plt.scatter(n_pixels, peak_arr, s = 10)
plt.xlabel("Pixel number")
plt.ylabel("Time sample of the peak")
plt.title("Peak time for every pixel in dl0 data")
#plt.ylim(10,30)
plt.savefig(f"/Users/vdk/PeakTime_DL0_event{str(k)+str(0)+str(event_number)}")

In [ ]:
t = histplot(peak_arr, bins = 40)
plt.xlabel("time sample")
plt.title("Hist of peak times for all pixels in dl1")
print(t)

In [ ]:
event.dl0.tel[teln].waveform[0]

In [ ]:
np.sum(event.r1.tel[teln].waveform, axis = 0)

In [ ]:
np.max(event.r1.tel[teln].waveform, axis = 0)

In [ ]:
plt.plot(n_timesamples, np.max(event.r1.tel[teln].waveform, axis = 0))

In [ ]:
plt.plot(n_timesamples, np.sum(event.r1.tel[teln].waveform, axis = 0))

In [ ]:
event.dl1.tel[teln].parameters

In [ ]:
event.dl1.tel[teln].parameters.peak_time_statistics

In [ ]:
event.dl1.tel[teln].parameters.morphology.n_islands

In [ ]:
###
###
###
###
###

k = [2,2,9,25,32,22,22,32]
event_number = [223, 438,222,51,64,126,365,298]
tel_arr = [2,3,2,1,1,1,1,3]
work_range = [0,1,2,3,4,5,6,7]

for_hist = []

#filename = f'/Users/vdk/sim_run_folder/output_prod_v12/corsika_theta_28.4_az_182.75_run{k}.simtel.gz'
#source = EventSource(filename, max_events=9000)
#event_iterator = iter(source)

for i in work_range:
    filename = f'/Users/vdk/sim_run_folder/output_prod_v12/corsika_theta_28.4_az_182.75_run{k[i]}.simtel.gz'
    source = EventSource(filename, max_events=9000)
    event_iterator = iter(source)
    n_pixels = [i for i in range(1855)]
    n_timesamples = [i for i in range(40)]
    for n,j in enumerate(event_iterator):
        if n == event_number[i]:
            teln = tel_arr[i]
            #image_processor = ImageProcessor(source.subarray)
            #muon_processor = MuonProcessor(source.subarray)
            #calib = CameraCalibrator(image_extractor_type="GlobalPeakWindowSum",subarray = source.subarray)
            #calib(j)
            #image_processor(j)
            #muon_processor(j)
            event = j
            #fig, axes = plt.subplots(2, 1, sharex=False, gridspec_kw={'height_ratios': [1, 1]}, figsize=(9,12))
            #fig.subplots_adjust(hspace=0.2)
            #ax = axes[0]

            
            #plt.scatter(n_pixels,event.dl1.tel[teln].peak_time, s = 6, alpha = 0.5)
            peak_arr = np.empty(1855, dtype = int)
            np.argmax(event.r1.tel[teln].waveform, axis = 1, out = peak_arr)
            plt.scatter(n_pixels,peak_arr, s = 6, alpha = 0.5)
            
            
            #camgeom = source.subarray.tel[1].camera.geometry
            #title=f"event{event_number[i]}_fit{round(j.muon.tel[teln].efficiency.optical_efficiency,3)}_width{round(j.muon.tel[teln].efficiency.width.to_value(),3)}"
            #disp = CameraDisplay(camgeom,title=title, ax = ax)
            #disp.image = j.simulation.tel[teln].true_image #.sum(axis=1)
            #disp.cmap = plt.cm.RdBu_r
            #disp.add_colorbar()
            #disp.set_limits_percent(95)
            for_hist = np.append(for_hist, peak_arr) 
            #for pix_id in range(0,1855):
                #bx.plot(j.r0.tel[teln].waveform[0][pix_id], label="pix {}".format(pix_id), drawstyle='steps')
            #    bx.plot(j.r1.tel[teln].waveform[pix_id], label="pix {}".format(pix_id), drawstyle='steps')
            #bx.set_ylabel("ADC counts")
            #bx.set_xlabel("Samples")
            #bx.set_title("Waveforms")
            #plt.scatter(n_pixels,event.dl1.tel[teln].peak_time, s = 6, alpha = 0.5)
            #plt.title("Peak time for every pixel in dl1 data")
            #plt.savefig(f"/Users/vdk/PeakTime_DL1_event{str(k)+str(0)+str(event_number)}")
            break


plt.xlabel("Pixel number")
plt.ylabel("Time sample of the peak")
plt.title("Peak time for every pixel in 8 events (dl0 data)")
plt.show()


In [ ]:
histplot(for_hist, alpha = 0.8, binrange=[0,40], bins = 40)
plt.xlabel("Time sample")
plt.ylabel("Count of pixels with appropriate peaktime")
plt.title("Histogram for peaktime of 8 events (dl1 data)")
#plt.savefig(f"/Users/vdk/Histplot_PeakTime_DL1events")

In [ ]:
for_hist